<a href="https://colab.research.google.com/github/Chinna-Anaparthi/GPU/blob/main/GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

# Check available GPUs
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Check if TensorFlow is using a GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"TensorFlow is using the following GPU: {tf.test.gpu_device_name()}")
else:
    print("TensorFlow is not using a GPU.")


Num GPUs Available:  1
TensorFlow is using the following GPU: /device:GPU:0


In [ ]:
import torch

print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Number of GPU:  1
GPU Name:  Tesla T4
Using device: cuda


In [ ]:
print("hi")

In [ ]:
!pip install fastapi  uvicorn python-dotenv google.generativeai python-multipart nest_asyncio

In [ ]:
from fastapi import FastAPI, HTTPException, Form
# from vertexai.generative_models import GenerativeModel
import google.generativeai as genai
import os
import uvicorn
import nest_asyncio
from fastapi.middleware.cors import CORSMiddleware
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"serverKey.json"
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))



# Initialize chat session and history globally
chat_session = None

# Function to generate the response
def get_gemini_response(user_prompt: str):
    global chat_session
    try:
        if chat_session is None:
            model =genai.GenerativeModel(model_name="gemini-1.5-flash-002", system_instruction="system instruction")
            chat_session = model.start_chat()

        generation_config = {
            "temperature": 1,
            "top_p": 0.95,
            "max_output_tokens": 8192,
        }

        final_result = ""
        prompt = f"User's original request: {user_prompt}"

        response = chat_session.send_message(prompt, generation_config=generation_config)
        final_result = response.text  # Keep updating with the latest output


        print("final_result:", final_result)
        return final_result

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error generating text: {str(e)}")

# API endpoint for processing the user prompt
@app.post("/generate_response/")
async def generate_response(user_prompt: str = Form(...)):
    try:
        response_text = get_gemini_response(user_prompt)
        return {"response": response_text}

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


if __name__ == "__main__":
    nest_asyncio.apply()
    try:
        uvicorn.run(app, host="127.0.0.1", port=8000)
    except KeyboardInterrupt:
        print("Server stopped by user.")


INFO:     Started server process [246]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
